## PICKLE & FLASK

### PERSISTING A MODEL WITH PICKLE

In [219]:
import pickle

In [285]:
lr_model_path = os.path.join(os.path.pardir, 'models', 'lrModel.pkl') #'../models/lrModel.pkl'
scalar_path = os.path.join(os.path.pardir, 'models', 'scalar.pkl') #'../models/scalar.pkl'

In [297]:
# Open files for write
model_pickle = open(lr_model_path, 'wb')
scalar_pickle = open(scalar_path, 'wb')

In [298]:
pickle.dump(lrModel, model_pickle)
pickle.dump(X_train_scaled2, scalar_pickle)

In [299]:
model_pickle.close()
scalar_pickle.close()

In [302]:
!ls -l ../models

total 1264
-rw-r--r--  1 ganeshpillai  staff  480096 Sep 20 19:33 lrModel.pkl
-rw-r--r--  1 ganeshpillai  staff     383 Sep 21 08:22 lrModel_new.pkl
-rw-r--r--  1 ganeshpillai  staff  159649 Sep 21 08:22 scalar.pkl


#### READ PERSISTED MODEL

In [308]:
# Open files for read
model_pickle_r = open(lr_model_path, 'rb')
scalar_pickle_r = open(scalar_path, 'r')

In [309]:
lrModelLoaded = pickle.load(model_pickle_r)

In [372]:
model_pickle_r.close()

In [311]:
type(lrModelLoaded)

sklearn.linear_model.logistic.LogisticRegression

### HELLO WORLD USING FLASK

In [312]:
import os
hello_world_script_file = os.path.join(os.path.pardir, 'src', 'models', 'hello_world_api.py')

In [329]:
%%writefile $hello_world_script_file
from flask import Flask, request

app = Flask(__name__)

@app.route('/api' , methods=['POST'])
def say_hello():
    data = request.get_json(force=True)
    name = data['name']
    return "hello" + name

if __name__ == '__main__':
    app.run(port=10002, debug = True)

Overwriting ../src/models/hello_world_api.py


In [330]:
!ls -l ../src/models

total 8
-rw-r--r--  1 ganeshpillai  staff    0 Sep  9 16:51 __init__.py
-rw-r--r--  1 ganeshpillai  staff  267 Sep 21 09:27 hello_world_api.py
-rw-r--r--  1 ganeshpillai  staff    0 Sep  9 16:51 predict_model.py
-rw-r--r--  1 ganeshpillai  staff    0 Sep  9 16:51 train_model.py


In [332]:
!python ../src/models/hello_world_api.py

 * Serving Flask app "hello_world_api" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:10002/ (Press CTRL+C to quit)
 * Restarting with stat
/anaconda3/bin/python: can't open file '../src/models/hello_world_api.py': [Errno 2] No such file or directory


In [333]:
import json
import requests

In [336]:
url = 'http://127.0.0.1:10002/api'
data = json.dumps({'name':'GANESH PILLAI'})
r = requests.post(url, data)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=10002): Max retries exceeded with url: //api (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a1a856358>: Failed to establish a new connection: [Errno 61] Connection refused',))

### TO DO: URL NOT WORKING. FIX THIS

## ML API USING FLASK

In [338]:
import os
machine_learning_flask_script_file = os.path.join(os.path.pardir, 'src', 'models', 'machine_learning_flask_api.py')

In [339]:
%%writefile $machine_learning_flask_script_file
from flask import Flask, request
import pandas as pd
import numpy as np
import json
import pickle
import os

app = Flask(__name__)

lr_model_path = os.path.join(os.path.pardir, 'models', 'lrModel.pkl') #'../models/lrModel.pkl'
model_pickle_r = open(lr_model_path, 'rb')
lrModelLoaded = pickle.load(model_pickle_r)

# columns
columns = [ u'Age', u'Fare', u'FamilySize', \
       u'IsMother', u'IsMale', u'Deck_A', u'Deck_B', u'Deck_C', u'Deck_D', \
       u'Deck_E', u'Deck_F', u'Deck_G', u'Deck_Z', u'Pclass_1', u'Pclass_2', \
       u'Pclass_3', u'Title_Lady', u'Title_Master', u'Title_Miss', u'Title_Mr', \
       u'Title_Mrs', u'Title_Officer', u'Title_Sir', u'Fare_Bin_very_low', \
       u'Fare_Bin_low', u'Fare_Bin_high', u'Fare_Bin_very_high', u'Embarked_C', \
       u'Embarked_Q', u'Embarked_S', u'AgeState_Adult', u'AgeState_Child'] 


@app.route('/api' , methods=['POST'])
def make_prediction():
    # read json object and convert to json string
    data = json.dumps(request.get_json(force=True))
    df = pd.read_json(data)
    #extract passenger ids
    passenger_ids = df['PassengerId'].ravel()
    survived = df['Survived'].ravel()
    X = df[columns].values().astype('float')
    predictions = lrModelLoaded.predict(X_scaled)
    
    df_response = pd.DataFrame({'PassengerId': passenger_ids, 'Predicted': predictions, 'Actual': survived})
    return df_response.to_json()

if __name__ == '__main__':
    app.run(port=10002, debug = True)

Writing ../src/models/machine_learning_flask_api.py


In [340]:
!python ../src/models/machine_learning_flask_api.py

 * Serving Flask app "machine_learning_flask_api" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:10002/ (Press CTRL+C to quit)
 * Restarting with stat
/anaconda3/bin/python: can't open file '../src/models/machine_learning_flask_api.py': [Errno 2] No such file or directory


In [341]:
#Invoking API using requests
processed_data_processed_data_path = os.path.join(os.path.pardir, 'data', 'processed')  
train_data_path = os.path.join(processed_data_path, 'train.csv') 
train_df = pd.read_csv(train_data_path)

In [343]:
survived_passengers = train_df[train_df['Survived'] == 1][:5]

In [344]:
survived_passengers

,PassengerId,Survived,Adult_No,Adult_Yes,Age,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,...,Pclass_1,Pclass_2,Pclass_3,Title_lady,Title_master,Title_miss,Title_mr,Title_mrs,Title_officer,Title_sir
1,2,1,0,1,38.0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
2,3,1,0,1,26.0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3,4,1,0,1,35.0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
8,9,1,0,1,27.0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
9,10,1,1,0,14.0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [349]:
make_api_request(survived_passengers.to_json())

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=10002): Max retries exceeded with url: /api (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a1a87cda0>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [350]:
result = make_api_request(train_df.to_json())
df_result = pd.read_json(json.dumps(result))
df_result.head()

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=10002): Max retries exceeded with url: /api (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a1a864dd8>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [351]:
np.mean(df_result.Actual == df_result.Predicted)

NameError: name 'df_result' is not defined

In [352]:
def make_api_request(data):
    url = 'http://localhost:10002/api'
    r = requests.post(url, data)
    return r.json()

#### TO DO: FIX THIS

In [3]:
# END